In [1]:
import nltk
import numpy as np
import random
import string # to process standard python strings

In [2]:
f=open('ppd.txt','r',errors = 'ignore')
raw=f.read()
raw=raw.lower()# converts to lowercase

In [3]:
# nltk.download('punkt') # first-time use only
# nltk.download('wordnet') # first-time use only

In [4]:
sent_tokens = nltk.sent_tokenize(raw)# converts to list of sentences 
word_tokens = nltk.word_tokenize(raw)# converts to list of words

In [5]:
sent_tokens[:2]

['postpartum depression (ppd), also called postnatal depression, is a type of mood disorder associated with childbirth, which can affect both sexes.symptoms may include extreme sadness, low energy, anxiety, crying episodes, irritability, and changes in sleeping or eating patterns.onset is typically between one week and one month following childbirth.ppd can also negatively affect the newborn child.',
 "while the exact cause of ppd is unclear, the cause is believed to be a combination of physical, emotional, genetic, and social factors.these may include factors such as hormonal changes and sleep deprivation.risk factors include prior episodes of postpartum depression, bipolar disorder, a family history of depression, psychological stress, complications of childbirth, lack of support, or a drug use disorder.diagnosis is based on a person's symptoms."]

In [6]:
word_tokens[:2]

['postpartum', 'depression']

In [7]:
lemmer = nltk.stem.WordNetLemmatizer()
#WordNet is a semantically-oriented dictionary of English included in NLTK.
def LemTokens(tokens):
    return [lemmer.lemmatize(token) for token in tokens]
remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [8]:
GREETING_INPUTS = ("hello", "hi", "greetings", "sup", "what's up","hey",)
GREETING_RESPONSES = ["hi", "hey", "*nods*", "hi there", "hello", "I am glad! You are talking to me"]
def greeting(sentence):
 
    for word in sentence.split():
        if word.lower() in GREETING_INPUTS:
            return random.choice(GREETING_RESPONSES)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [10]:
def response(user_response):
    robo_response=''
    sent_tokens.append(user_response)
    TfidfVec = TfidfVectorizer(tokenizer=LemNormalize, stop_words = 'english')
    tfidf = TfidfVec.fit_transform(sent_tokens)
    vals = cosine_similarity(tfidf[-1], tfidf)
    idx=vals.argsort()[0][-2]
    flat = vals.flatten()
    flat.sort()
    req_tfidf = flat[-2]
    if(req_tfidf==0):
        robo_response=robo_response+"I am sorry! I don't understand you"
        return robo_response
    else:
        robo_response = robo_response+sent_tokens[idx]
        return robo_response

In [11]:
from flask import Flask
from flask import request
from twilio.twiml.messaging_response import MessagingResponse

app = Flask(__name__)


@app.route('/bot’, methods=['POST'])
def bot():
    flag=True
    print("ROBO: My name is Robo. I will answer your queries about Postpartum depression. If you want to exit, type Bye!")
    while(flag==True):
        user_response = request.values.get('msg', '')
        user_response=user_response.lower()
        robo_response = MessagingResponse()
        msg = robo_response.message()
        if(user_response!='bye'):
            if(user_response=='thanks' or user_response=='thank you' ):
                flag=False
                msg.body("ROBO: You are welcome..")
            else:
                if(greeting(user_response)!=None):
                    msg.body("ROBO: "+greeting(user_response))
                else:
                    msg.body("ROBO: ",end="")
                    msg.body(response(user_response))
                    sent_tokens.remove(user_response)
        else:
            flag=False
            msg.body("ROBO: Bye! take care..")
    return str(resp)

ROBO: My name is Robo. I will answer your queries about Postpartum depression. If you want to exit, type Bye!
Hello
ROBO: I am glad! You are talking to me
what is baby blues
ROBO: postpartum blues, commonly known as "baby blues," is a transient postpartum mood disorder characterized by milder depressive symptoms than postpartum depression.
behavioural symptoms?
ROBO: [54] symptoms typically resolve within two weeks.
what are the causes?
ROBO: profound lifestyle changes that are brought about by caring for the infant are also frequently hypothesized to cause ppd.
bye
ROBO: Bye! take care..
